In [2]:
import requests
import json
from bs4 import BeautifulSoup as bs
import pandas as pd
import re

In [3]:
#Get top shoe IDs
url = 'https://2fwotdvm2o-dsn.algolia.net/1/indexes/product_variants_v2/query?x-algolia-agent=Algolia for vanilla JavaScript 3.25.1&x-algolia-application-id=2FWOTDVM2O&x-algolia-api-key=ac96de6fef0e02bb95d433d8d5c7038a'
data = {'distinct':'true', 'hitsPerPage':'50','page':'19', 'query': ''}
r = requests.post(url, data=json.dumps(data))
shoeList = pd.DataFrame.from_records(r.json()['hits'])
shoeList = shoeList[['product_template_id', 'name', 'main_picture_url']]
shoeList.rename(columns={'product_template_id': 'id', 'main_picture_url': 'picture'}, inplace=True)
shoeList

,id,name,picture
0,391425,Pharrell x NMD Human Race 'Inspiration Pack',https://image.goat.com/750/attachments/product...
1,306819,Balenciaga Triple S Trainer 'White' 2018,https://image.goat.com/750/attachments/product...
2,677042,Yeezy 700 V3 Kids 'Arzareth',https://image.goat.com/750/attachments/product...
3,19607,Air Force 1 Supreme Plyrs 'Players',https://image.goat.com/750/attachments/product...
4,656412,Supreme x Sk8-Hi 'Black Hole Punch Denim',https://image.goat.com/750/attachments/product...
5,176660,Air Foamposite One 'Eggplant' 2017,https://image.goat.com/750/attachments/product...
6,13602,Air Jordan 3 Retro 'Fire Red' 2013,https://image.goat.com/750/attachments/product...
7,13729,Air Jordan 3 Retro 'True Blue' 2011,https://image.goat.com/750/attachments/product...
8,479524,Air Jordan 4 Retro 'Laser',https://image.goat.com/750/attachments/product...
9,13617,Air Jordan 5 Retro 'Fire Red' 2013,https://image.goat.com/750/attachments/product...


In [4]:
shoeDict = {}
for _, shoe in shoeList.iterrows():
    print('Processing', shoe['name'])
    #Get shoe info
    url = 'https://www.goat.com/web-api/v1/product_variants'
    param = {'productTemplateId':shoe['id']} 
    header = {'User-Agent': 'Mozilla/5.0'}
    req = requests.get(url, headers=header, params=param)
    jsonInfo = req.json()
    #Turn to dataframe and clean up
    s = pd.DataFrame.from_records(jsonInfo)
    s.drop(['sizeOption', 'lowestPriceCentsByWarehouse', 'instantShipLowestPriceCents'], axis=1, inplace=True)
    s.rename(columns={'lowestPriceCents': 'price'}, inplace=True)
    s['price'] = [re.findall('(\d+)00', str(x))[0] for x in s['price']]
    shoeDict[shoe['name']] = s

Processing Pharrell x NMD Human Race 'Inspiration Pack'
Processing Balenciaga Triple S Trainer 'White' 2018
Processing Yeezy 700 V3 Kids 'Arzareth'
Processing Air Force 1 Supreme Plyrs 'Players'
Processing Supreme x Sk8-Hi 'Black Hole Punch Denim'
Processing Air Foamposite One 'Eggplant' 2017
Processing Air Jordan 3 Retro 'Fire Red' 2013
Processing Air Jordan 3 Retro 'True Blue' 2011
Processing Air Jordan 4 Retro 'Laser'
Processing Air Jordan 5 Retro 'Fire Red' 2013
Processing Wmns Air VaporMax 2020 Flyknit 'Light Arctic Pink'
Processing Air More Uptempo 'Black White' 2020
Processing Air Jordan 4 Retro 'Military Blue' 2006
Processing LeBron 7 'MVP'
Processing Air Jordan 12 Retro 'Dark Grey'
Processing Yeezy Boost 350 'Moonrock'
Processing Travis Scott x Air Max 270 React ENG PS 'Cactus Trails'
Processing Air Jordan 11 BG 'Legend Blue'
Processing Air Jordan 12 Retro GS 'Gym Red'
Processing JJJJound x 992 'Grey'
Processing Sacai x Blazer Mid 'Black Blue'
Processing Kasina x Dunk Low 'Roa

In [7]:
for name, df in shoeDict.items():
    print(name)
    p = df[(df.shoeCondition != 'new_no_defects')]
    display(p)

Air Jordan 6 Retro SP 'Reflections Of A Champion'


,size,shoeCondition,boxCondition,price
8,8.5,used,good_condition,120
10,9.0,used,good_condition,159
12,9.5,used,good_condition,160
13,9.5,used,no_original_box,155
14,9.5,new_with_defects,good_condition,250
15,10.0,used,good_condition,100
16,10.0,used,badly_damaged,75
20,10.5,used,missing_lid,181
22,10.5,used,good_condition,100
23,10.5,new_with_defects,good_condition,205


Pharrell x NMD Human Race Trail 'Solar Pack'


,size,shoeCondition,boxCondition,price
0,4.0,used,good_condition,230
6,7.0,used,no_original_box,260
8,7.5,used,good_condition,227
11,8.0,used,good_condition,160
12,8.0,used,no_original_box,245
13,8.0,used,badly_damaged,160
17,8.5,used,good_condition,185
18,8.5,used,no_original_box,200
19,8.5,used,badly_damaged,135
20,9.0,new_with_defects,good_condition,133


LeBron 18 'Reflections'


...

,size,shoeCondition,boxCondition,price
4,5.0,used,good_condition,65
10,6.5,used,good_condition,160
14,7.5,used,no_original_box,120
15,7.5,new_with_defects,good_condition,195
18,8.0,used,good_condition,175
19,8.0,used,no_original_box,120
21,8.5,new_with_defects,good_condition,150
22,8.5,used,good_condition,110
24,9.0,used,good_condition,186
26,9.5,used,no_original_box,130


Off-White x Hyperdunk 2017 Flyknit 'The Ten'


,size,shoeCondition,boxCondition,price
0,5.0,used,badly_damaged,455
4,7.0,used,no_original_box,375
6,7.0,used,good_condition,750
7,7.0,used,badly_damaged,350
8,7.5,used,no_original_box,499
9,7.5,new_with_defects,badly_damaged,950
10,7.5,used,badly_damaged,665
12,8.0,new_with_defects,badly_damaged,890
14,8.0,goat_clean,badly_damaged,575
15,8.0,used,badly_damaged,900


Dior B23 High 'Dior Oblique - Red'


,size,shoeCondition,boxCondition,price


In [8]:
#Get shoe info
url = 'https://www.goat.com/web-api/v1/product_variants'
header = {'User-Agent': 'Mozilla/5.0'}
for index, row in shoeList.iterrows():
    param = {'productTemplateId':row['id']} 
    r = requests.get(url, headers=header, params=param)
    jsonInfo = r.json()

In [116]:
df

,size,shoeCondition,boxCondition,price
0,7.0,new_no_defects,good_condition,44500
1,7.5,new_no_defects,good_condition,46000
2,8.0,new_no_defects,good_condition,38900
3,8.5,new_no_defects,good_condition,37500
4,9.0,new_no_defects,good_condition,36900
5,9.0,new_no_defects,no_original_box,75000
6,9.5,new_no_defects,good_condition,37400
7,9.5,new_with_defects,no_original_box,62700
8,9.5,new_with_defects,good_condition,34500
9,10.0,new_no_defects,good_condition,37400


In [102]:
url = 'https://2fwotdvm2o-dsn.algolia.net/1/indexes/*/queries?x-algolia-agent=Algolia%20for%20vanilla%20JavaScript%20(lite)%203.25.1%3BJS%20Helper%20(3.1.2)%3Breact%20(16.13.1)%3Breact-instantsearch%20(6.5.0)&x-algolia-application-id=2FWOTDVM2O&x-algolia-api-key=ac96de6fef0e02bb95d433d8d5c7038a'
req = requests.post(url, headers=header)
req

<Response [400]>

In [106]:
url = 'https://2fwotdvm2o-dsn.algolia.net/1/indexes/product_variants_v2/query?x-algolia-agent=Algolia for vanilla JavaScript 3.25.1&x-algolia-application-id=2FWOTDVM2O&x-algolia-api-key=ac96de6fef0e02bb95d433d8d5c7038a'
data = {"params":"distinct=true&facetFilters=()&facets=%5B%22size%22%5D&hitsPerPage=20&numericFilters=%5B%5D&page=0&query="}
r = requests.post(url, data=json.dumps(data))

print(r.json()['hits'][0])

{'product_template_id': 612859, 'shoe_condition': 'new_no_defects', 'box_condition': 'missing_lid', 'lowest_price_cents': 21500, 'instant_ship_lowest_price_cents': None, 'activity': None, 'brand_name': 'Air Jordan', 'breadcrumb_path_url': None, 'category': ['lifestyle'], 'color': 'Black', 'designer': 'Tinker Hatfield', 'details': 'Black/Black/Dark Concord', 'gender': ['men'], 'grid_glow_picture_url': 'https://image.goat.com/glow-4-5-25/375/attachments/product_template_pictures/images/044/294/657/original/612859_00.png.png', 'grid_picture_url': 'https://image.goat.com/375/attachments/product_template_pictures/images/044/294/657/original/612859_00.png.png', 'has_picture': True, 'has_under_retail_availability': False, 'keywords': [], 'main_glow_picture_url': 'https://image.goat.com/glow-4-5-25/750/attachments/product_template_pictures/images/044/294/657/original/612859_00.png.png', 'main_picture_url': 'https://image.goat.com/750/attachments/product_template_pictures/images/044/294/657/ori

In [110]:
url = 'https://2fwotdvm2o-dsn.algolia.net/1/indexes/product_variants_v2/query?x-algolia-agent=Algolia for vanilla JavaScript 3.25.1&x-algolia-application-id=2FWOTDVM2O&x-algolia-api-key=ac96de6fef0e02bb95d433d8d5c7038a'
data = {"params":"distinct=true&facetFilters=()&facets=%5B%22size%22%5D&hitsPerPage=20&numericFilters=%5B%5D&page=0&query="}
r = requests.post(url, data=json.dumps(data))
print('1')
df = pd.DataFrame.from_records(r.json()['hits'])
df

1


,product_template_id,shoe_condition,box_condition,lowest_price_cents,instant_ship_lowest_price_cents,activity,brand_name,breadcrumb_path_url,category,color,...,retail_price_cents_twd,lowest_price_cents_jpy,instant_ship_lowest_price_cents_jpy,retail_price_cents_jpy,lowest_price_cents_krw,instant_ship_lowest_price_cents_krw,retail_price_cents_krw,product_taxonomy,objectID,_highlightResult
0,612859,new_no_defects,missing_lid,21500,NaN,None,Air Jordan,None,[lifestyle],Black,...,545300,2274100,NaN,2009700,24577500,NaN,21719700,{},10508210,"{'brand_name': {'value': 'Air Jordan', 'matchL..."
1,695243,new_no_defects,no_original_box,24900,NaN,None,adidas,None,[lifestyle],White,...,635900,2631000,NaN,2324600,28435500,NaN,25123700,{},10486691,"{'brand_name': {'value': 'adidas', 'matchLevel..."
2,567948,new_no_defects,good_condition,27900,NaN,None,Air Jordan,None,[lifestyle],Brown,...,487900,2951100,NaN,1798200,31893600,NaN,19433400,{},10060388,"{'brand_name': {'value': 'Air Jordan', 'matchL..."
3,662236,new_no_defects,good_condition,53900,64300.0,None,Air Jordan,None,[lifestyle],Cream,...,645700,5701200,6801200.0,2379900,61615300,73503900.0,25720700,{},10406445,"{'brand_name': {'value': 'Air Jordan', 'matchL..."
4,664578,new_no_defects,good_condition,52000,62800.0,None,Air Jordan,None,[lifestyle],Blue,...,717500,5500200,6642500.0,2644400,59443300,71789200.0,28578500,{},9685694,"{'brand_name': {'value': 'Air Jordan', 'matchL..."
5,568627,new_no_defects,good_condition,22700,29600.0,None,Air Jordan,None,[lifestyle],Green,...,487900,2401100,3130900.0,1798200,25949300,33837000.0,19433400,{},9889147,"{'brand_name': {'value': 'Air Jordan', 'matchL..."
6,631937,new_no_defects,badly_damaged,25000,NaN,None,adidas,None,[lifestyle],Black,...,631400,2644400,NaN,2327000,28578500,NaN,25149100,{},10169542,"{'brand_name': {'value': 'adidas', 'matchLevel..."
7,695795,new_no_defects,good_condition,19200,25800.0,None,Nike,None,[lifestyle],Orange,...,373100,2030900,2729000.0,1375100,21948300,29493000.0,14860900,{},10360037,"{'brand_name': {'value': 'Nike', 'matchLevel':..."
8,519961,new_no_defects,good_condition,21000,NaN,lifestyle,Air Jordan,None,[lifestyle],Blue,...,491400,2218900,NaN,1796300,23981800,NaN,19413800,{},8052924,"{'activity': {'value': 'lifestyle', 'matchLeve..."
9,603678,new_no_defects,no_original_box,100000,NaN,None,Air Jordan,None,[lifestyle],Tan,...,574000,10577300,NaN,2115500,114314000,NaN,22862800,{},8989502,"{'brand_name': {'value': 'Air Jordan', 'matchL..."


In [113]:
grid_picture_url

0    https://image.goat.com/attachments/collection_...
1    https://image.goat.com/attachments/collection_...
2    https://image.goat.com/attachments/collection_...
3    https://image.goat.com/attachments/collection_...
Name: web_picture, dtype: object

In [136]:
shoeList.drop(['shoe_condition', 'box_condition','lowest_price_cents', 'instant_ship_lowest_price_cents',
              'activity', 'breadcrumb_path_url', 'category', 'color', 'designer', 'details', 'gender',
              'grid_glow_picture_url', 'grid_picture_url', 'has_picture', 'has_under_retail_availability',
              'keywords', 'main_glow_picture_url', 'main_picture_url', 'midsole'], axis=1).head(3)


,product_template_id,brand_name,name,nickname,original_picture_url,product_category,product_type,release_date,release_date_name,retail_price_cents,...,retail_price_cents_twd,lowest_price_cents_jpy,instant_ship_lowest_price_cents_jpy,retail_price_cents_jpy,lowest_price_cents_krw,instant_ship_lowest_price_cents_krw,retail_price_cents_krw,product_taxonomy,objectID,_highlightResult
0,612859,Air Jordan,Air Jordan 12 Retro 'Dark Concord',Dark Concord,https://image.goat.com/attachments/product_tem...,shoes,sneakers,2020-10-23T23:59:59.999Z,,19000,...,545300,2274100,NaN,2009700,24577500,NaN,21719700,{},10508210,"{'brand_name': {'value': 'Air Jordan', 'matchL..."
1,695243,adidas,Yeezy Boost 350 V2 'Natural',Natural,https://image.goat.com/attachments/product_tem...,shoes,sneakers,2020-10-24T23:59:59.999Z,,22000,...,635900,2631000,NaN,2324600,28435500,NaN,25123700,{},10486691,"{'brand_name': {'value': 'adidas', 'matchLevel..."
2,567948,Air Jordan,Air Jordan 1 Retro High OG 'Dark Mocha',Dark Mocha,https://image.goat.com/attachments/product_tem...,shoes,sneakers,2020-10-31T07:00:00.000Z,,17000,...,487900,2951100,NaN,1798200,31893600,NaN,19433400,{},10060388,"{'brand_name': {'value': 'Air Jordan', 'matchL..."


,product_template_id,name,main_picture_url
0,612859,Air Jordan 12 Retro 'Dark Concord',https://image.goat.com/750/attachments/product...
1,695243,Yeezy Boost 350 V2 'Natural',https://image.goat.com/750/attachments/product...
2,567948,Air Jordan 1 Retro High OG 'Dark Mocha',https://image.goat.com/750/attachments/product...
3,662236,Off-White x Air Jordan 5 SP 'Sail',https://image.goat.com/750/attachments/product...
4,664578,Union LA x Air Jordan 4 Retro 'Off Noir',https://image.goat.com/750/attachments/product...
5,568627,Wmns Air Jordan 1 Retro High OG 'Lucky Green',https://image.goat.com/750/attachments/product...
6,631937,Yeezy Boost 350 V2 'Carbon',https://image.goat.com/750/attachments/product...
7,695795,Air Force 1 Low 'Orange Skeleton',https://image.goat.com/750/attachments/product...
8,519961,Air Jordan 1 Retro High OG 'Royal Toe',https://image.goat.com/750/attachments/product...
9,603678,Off-White x Wmns Air Jordan 4 SP 'Sail',https://image.goat.com/750/attachments/product...


In [139]:
shoeList['grid_picture_url']

0     https://image.goat.com/375/attachments/product...
1     https://image.goat.com/375/attachments/product...
2     https://image.goat.com/375/attachments/product...
3     https://image.goat.com/375/attachments/product...
4     https://image.goat.com/375/attachments/product...
5     https://image.goat.com/375/attachments/product...
6     https://image.goat.com/375/attachments/product...
7     https://image.goat.com/375/attachments/product...
8     https://image.goat.com/375/attachments/product...
9     https://image.goat.com/375/attachments/product...
10    https://image.goat.com/375/attachments/product...
11    https://image.goat.com/375/attachments/product...
12    https://image.goat.com/375/attachments/product...
13    https://image.goat.com/375/attachments/product...
14    https://image.goat.com/375/attachments/product...
15    https://image.goat.com/375/attachments/product...
16    https://image.goat.com/375/attachments/product...
17    https://image.goat.com/375/attachments/pro